# **개발 환경 구축**

**구글 드라이브 마운트**

- 개인 구글 드라이브의 저장소를 Colab에서 사용하기 위한 설정

In [13]:
from google.colab import drive
import os

if os.path.exists('/content/gdrive')==False:
  drive.mount('/content/gdrive')
  print('Google Drive is mounted\n')
else:
  print('Google Drive is already mounted\n')

Google Drive is already mounted



**KoNLPy 한국어 처리 패키지 설치**

- Komoran 형태소 분석기 사용을 위해 설치

In [14]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 256 kB in 2s (150 kB/s)
Reading package lists... Done
Reading package lists... Done
B

# **데이터 전처리**

In [0]:
import numpy as np
import codecs            #인코딩을 지정해 파일 읽고 쓰기
import re                #정규 표현식을 지원하는 re(regular expression) 모듈
import itertools         #itertools 모듈에는 반복 가능한 데이터 스트림을 처리하는 데 유용한 많은 함수와 제네레이터가 포함되어 있음
from collections import Counter        #리스트에 있는 각 항목의 갯수를 셀 수 있음
from konlpy.tag import Komoran         #KoNLPy는 한국어 정보처리를 위한 파이썬 패키지이며, 형태소 분석을 위해 Komoran 이용
from csv import DictReader
from csv import DictWriter

komoran = Komoran()

**Data 클래스 정의**

- csv 파일을 불러와서 제목, 본문, 가짜 뉴스 Label 단위로 저장하는 클래스

In [0]:
class Data:
    def __init__(self, file_instances):
      
        self.instances = self.read(file_instances)    # 내부 메소드인 read()를 통해 csv파일을 불러와서 리스트 형태로 저장
        
        # csv 파일을 읽어서 파일 내에 'title', 'content', 그리고 'Label'을 각각 key로 하여 데이터를 저장할 3개의 dictionary 자료형 선언 -> {key : value}
        self.headlines = {}
        self.bodies = {}
        self.labels = {}
        
        # 각 기사의 순번(seqid)을 정수형 데이터로 변환
        for instance in self.instances:
            instance['seqid'] = int(instance['seqid'])
        
        # headlines에 'title'열을 순서대로 저장
        for head in self.instances:
            self.headlines[head['seqid']] = head['title']
        
        # bodies에 'content'열을 순서대로 저장
        for body in self.instances:
            self.bodies[body['seqid']] = body['content']
        
        # labels에 'Label'열을 순서대로 저장
        for label in self.instances:
            self.labels[label['seqid']] = label['Label']
    
    # csv 파일을 읽는 메소드 선언
    def read(self, filename):
        rows = []
       
        with open(filename, "r") as table:
            r = DictReader(table)        # csv를 dictionary 타입으로 읽기
            
            for line in r:               # 한 줄씩 읽어서 각 라인을 rows 리스트에 넣기
                rows.append(line)          
        return rows
    
    # 전체 기사 데이터를 반환
    def get_data(self):
        return self.instances
      
      
#인코딩 문제로 Test를 위한 클래스 따로 설정
class Data_test:
    def __init__(self, file_instances):
      
        self.instances = self.read(file_instances)    # 내부 메소드인 read()를 통해 csv파일을 불러와서 리스트 형태로 저장
        
        # csv 파일을 읽어서 파일 내에 'title', 'content', 그리고 'Label'을 각각 key로 하여 데이터를 저장할 3개의 dictionary 자료형 선언 -> {key : value}
        self.headlines = {}
        self.bodies = {}
        self.labels = {}
        
        # 각 기사의 순번(seqid)을 정수형 데이터로 변환
        for instance in self.instances:
            instance['seqid'] = int(instance['seqid'])
        
        # headlines에 'title'열을 순서대로 저장
        for head in self.instances:
            self.headlines[head['seqid']] = head['title']
        
        # bodies에 'content'열을 순서대로 저장
        for body in self.instances:
            self.bodies[body['seqid']] = body['content']
        
        # labels에 'label'열을 순서대로 저장
        for label in self.instances:
            self.labels[label['seqid']] = label['label']
    
    # csv 파일을 읽는 메소드 선언
    def read(self, filename):
        rows = []
       
        with open(filename, "r", encoding='euc-kr') as table:
            r = DictReader(table)        # csv를 dictionary 타입으로 읽기
            
            for line in r:               # 한 줄씩 읽어서 각 라인을 rows 리스트에 넣기
                rows.append(line)          
        return rows
    
    # 전체 기사 데이터를 반환
    def get_data(self):
        return self.instances

**데이터 처리에 쓰이는 함수들 정의 1**

In [0]:
# 토큰화 : 주어진 코퍼스(corpus)에서 토큰(token)이라 불리는 단위로 나누는 작업을 토큰화(Tokenization)라고 함 -> 코퍼스 = 자연어 데이터, 토큰 = 상황에 따라 다르지만, 보통 의미있는 단위로 정의

def tokenize(string):      # 문자열을 받아서 의미있는 단위로 나누어 형태소를 붙여주는 함수
    
    # Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py

    final_string = ''
    
    # 크롤링 된 기사들의 본문 중에는 기사 중간에 연속적인 개행/공백 이 들어가 있거나, 심지어 크롤링을 실패하고 개행/공백 만 있는 경우가 있음
    # Komoran은 형태소를 분석할 때 이러한 개행, 공백 문자에 의해 에러가 발생하기에 제거를 해줘야 함
    string = "".join(string.splitlines())   # 문자열을 읽어 개행을 제거
    string = " ".join(string.split())       # 문자열을 읽어 중복되는 공백을 제거
    
    # 위 과정을 거쳐 반환된 문자열이 만약 공백이라면, 형태소 분석을 할 수 없으므로, 공백을 반환해 준다
    if string == '':
      return final_string
    
    # 문자열을 읽어 의미있는 단어들을 라벨링하여 리스트로 반환 -> 결과 예) ['한글NNP', '형태소NNP', '분석기NNG', '코모NNP', '테스트NNP', '중NNB', '이VCP']
    tokenized = [''.join(t) for t in komoran.pos(string) if t[1] in ['NNG','NNP','NNB','NR','VV','VA','VCP','VCN','XSV','XSA','SN','MAG','MM','MAJ']]
#not in ['ETM','EC','ETN','EF','EP','NF','NV','NA','SW','SO','SP','SF','SE','SS','IC','XSN','XPN']]

    # tokenized 리스트 원소들을 " " 간격으로 하나의 문자열로 합쳐서 반환
    for tokens in tokenized:
        final_string += tokens + " "
    return final_string

def flat(content):
    return ["{}{}".format(word, tag) for word, tag in komoran.pos(content)]

**데이터 처리에 쓰이는 함수들 정의 2**

In [0]:
def load_data_and_labels(file_instances):       # file_instances은 파일경로
    """
    데이터를 불러와서, 의미 있는 단어 단위로 나누고 라벨을 붙여서
    단어들의 리스트로 반환
    """
    
    #품사 태깅한 데이터를 저장할 리스트
    train_heads = []
    train_bodies = []
    train_labels = []
    
    # Data 객체 생성(파일 로드)
    data = Data(file_instances)
    
    # 크롤링 데이터 중에 간혹 제목은 있으나 본문이 없는 데이터가 있음. 이 경우 정확도를 낮추는 원인이 될 수 있으므로 빼줘야 함
    error_detect = 0  # 이 변수는 위와 같은 데이터가 있을 경우 카운트한다
 
    # for문을 통해 기사를 순서대로 하나씩 읽기
    for instance in data.instances:
        # 기사 넘버('seqid')
        news_id = instance['seqid']
        # 가짜 뉴스 판단 여부('Label')
        train_label = instance['Label']
        
        # 기사 본문을 읽어서 라벨링
        blank_detect = tokenize(data.bodies[news_id])
        if blank_detect == '':              # 만약 본문이 없어서 반환된 문자열이 없다면, error_detect에 1을 더하고 아래의 과정을 스킵한다
          error_detect += 1
          continue
          
        # 라벨링된 본문을 train_heads 리스트에 append
        train_bodies.append(blank_detect.strip())             # 각 기사마다 양 끝에 공백을 없애주기 위해 strip()
        
        # 기사 제목을 레벨링하여 저장
        train_heads.append(tokenize(data.headlines[news_id]).strip())

        # (기사 넘버, 판단 넘버)를 하나의 튜플로 리스트에 append
        train_labels.append((news_id - error_detect, train_label))  # news_id - error_detect 이 부분은 위에 말한대로 기사가 없어 그 순번을 건너뛰면
                                                                     # 인덱스가 그때마다 한칸식 건너가지게 되므로 에러 발생수만큼 빼준다
          
    # 모든 원소가 0인 (라벨링된 전체 기사 개수, 2) 크기의 다차원 배열 생성
    results = np.zeros((len(train_labels),2))

    # i : news_id, train_label : train_label
    for i, train_label in train_labels:        
      
        # one-hot 인코딩과 동일 과정이라 생각하면 됨
        if train_label == '1':
            results[i,1] = 1
        else:
            results[i,0] = 1

    return train_heads, train_bodies, results
  
  
def load_data_and_labels_test(file_instances):       # file_instances은 파일경로
    """
    데이터를 불러와서, 의미 있는 단어 단위로 나누고 라벨을 붙여서
    단어들의 리스트로 반환
    """
    
    #품사 태깅한 데이터를 저장할 리스트
    train_heads = []
    train_bodies = []
    train_labels = []
    
    # Data 객체 생성(파일 로드)
    data = Data_test(file_instances)
    
    # for문을 통해 기사를 순서대로 하나씩 읽기
    for instance in data.instances:
        # 기사 넘버('seqid')
        news_id = instance['seqid']
        # 가짜 뉴스 판단 여부('label')
        train_label = instance['label']
          
        # 라벨링된 본문을 train_heads 리스트에 append
        train_bodies.append(tokenize(data.bodies[news_id]).strip())             # 각 기사마다 양 끝에 공백을 없애주기 위해 strip()
        
        # 기사 제목을 레벨링하여 저장
        train_heads.append(tokenize(data.headlines[news_id]).strip())

        # (기사 넘버, 판단 넘버)를 하나의 튜플로 리스트에 append
        train_labels.append((news_id, train_label))
          
    # 모든 원소가 0인 (라벨링된 전체 기사 개수, 2) 크기의 다차원 배열 생성
    results = np.zeros((len(train_labels),2))

    # i : news_id, train_label : train_label
    for i, train_label in train_labels:        
      
        # one-hot 인코딩과 동일 과정이라 생각하면 됨
        if train_label == '1':
            results[i-1,1] = 1
        else:
            results[i-1,0] = 1

    return train_heads, train_bodies, results

**데이터 처리에 쓰이는 함수들 정의 3**

In [0]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):     # data : list(zip(x_train_head, x_train_body, y_train)
    '''
    Generates a batch iterator for a dataset.
    '''
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int(len(data)/batch_size) + 1       # epoch 당 batch 크기 결정
    
    # 각각의 epoch 마다 데이터를 shuffle 해준다
    for epoch in range(num_epochs):
        
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))     # np.random.permutation(np.arange(data_size))  : 0 ~ data_size-1까지(총 data_size개의 인덱스)를 무작위 순서의 배열로 반환
            shuffled_data = data[shuffle_indices]        # 반환된 무작위의 인덱스를 통해 데이터 shuffle    
        else:
            shuffled_data = data
            
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

# 라벨링한 데이터를 임베딩하는 함수
def load_word_embedding(file_name, vocab_processor, embedding_dim):  # file_name : 우리가 사용하는 것은 기존에 학습된 'fasttext_3_10.vec'

    initW = np.random.uniform(-0.25, 0.25, (len(vocab_processor.vocabulary_), embedding_dim))
    
    print("Load word2vec file {}\n".format(file_name))
    with open(file_name, "rb") as f:
        for idx, line in enumerate(f):
            word = []
            vectors = []
            if idx == 0:
                vocab_size, dim = line.strip().split()
            else:
                tks = line.strip().split()
                word = tks[0].strip().decode('utf8')
                vectors = tks[1:]
                idx = vocab_processor.vocabulary_.get(word)
                if idx != 0:
                    initW[idx] = np.array(vectors)
    print(initW)
    return initW

# **학습 모델(Convolutional Neural Networks)**

**Affine 클래스**
- Affine이란 순전파에서 수행하는 행렬의 내적을 기하학에서 부르는 말
- 퍼셉트론에서 배웠던 input값과 weight값의 행렬을 곱하고 거기에 편향(bias)를 더하여 최종적으로 출력값 Y를 반환하는 것과 같다.

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.rnn import GRUCell
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn

In [0]:
class Affine(object):
# Combine all the pooled features
    def __init__(
      self, sequence_length_head, sequence_length_body, num_classes, vocab_size_head, vocab_size_body,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.1):

        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        self.input_x_head = tf.placeholder(tf.int32, [None, sequence_length_head], name="input_x_head")
        self.input_x_body = tf.placeholder(tf.int32, [None, sequence_length_body], name="input_x_body")

        # Embedding layer
        self.embeddings_head = tf.Variable(
                tf.random_uniform([vocab_size_head, embedding_size], -1.0, 1.0),trainable=False)#trainable=false
        self.embedded_chars_head = tf.nn.embedding_lookup(self.embeddings_head, self.input_x_head)
        self.embedded_chars_expanded_head = tf.expand_dims(self.embedded_chars_head, -1)

        self.embeddings_body = tf.Variable(
                tf.random_uniform([vocab_size_body, embedding_size], -1.0, 1.0),trainable=False)#trainable=false
        self.embedded_chars_body = tf.nn.embedding_lookup(self.embeddings_body, self.input_x_body)
        self.embedded_chars_expanded_body = tf.expand_dims(self.embedded_chars_body, -1)

        # Create a convolution + maxpool layer for each filter size
        self.pooled_outputs_head = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-head-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, 256]
                W_head = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W_head")
                b_head = tf.Variable(tf.constant(0.1, shape=[256]), name="b_head")
                conv_head = tf.nn.conv2d(
                    self.embedded_chars_expanded_head,
                    W_head,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h_head = tf.nn.relu(tf.nn.bias_add(conv_head, b_head), name="relu_head")
                # Maxpooling over the outputs
                pooled_head = tf.nn.max_pool(
                    h_head,
                    ksize=[1, sequence_length_head - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                self.pooled_outputs_head.append(pooled_head)
                
                with tf.name_scope("dropout"):
                  self.h_drop = tf.nn.dropout(pooled_head, self.dropout_keep_prob)

        self.pooled_outputs_body = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-body-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, 1024]
                W_body = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W_body")
                b_body = tf.Variable(tf.constant(0.1, shape=[1024]), name="b_body")
                conv_body = tf.nn.conv2d(
                    self.embedded_chars_expanded_body,
                    W_body,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h_body = tf.nn.relu(tf.nn.bias_add(conv_body, b_body), name="relu_body")
                # Maxpooling over the outputs
                pooled_body = tf.nn.max_pool(
                    h_body,
                    ksize=[1, sequence_length_body - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                self.pooled_outputs_body.append(pooled_body)    
                with tf.name_scope("dropout"):
                  self.h_drop = tf.nn.dropout(pooled_body, self.dropout_keep_prob)

        l2_loss = tf.constant(0.0)

        pooled_outputs = tf.concat([self.pooled_outputs_head,self.pooled_outputs_body],-1,name='preconcat')
        print(pooled_outputs.shape)
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3, name='concat')
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
        print("여기")
        print(self.h_pool_flat.shape)

        W_fc1 = tf.Variable(tf.truncated_normal([1280,1024],stddev=0.1),name="W_fc1")
        b_fc1 = tf.Variable(tf.constant(0.1,shape=[1024]),name="b_fc1")
        h_fc1 = tf.nn.relu(tf.matmul(self.h_pool_flat,W_fc1) + b_fc1)
        print(h_fc1.shape)
        print("22222")

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(h_fc1, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[1024, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # CalculateMean cross-entropy loss
        with tf.name_scope("loss"):
            print(self.scores.shape)
            losses = tf.nn.softmax_cross_entropy_with_logits(logits = self.scores, labels = self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            print("%d/%d",self.predictions,self.input_y)
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

# **학습 코드**

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import learn
import codecs
import operator

**학습 parameter 설정**

In [0]:
embedding_dim=128
filter_sizes="3"
num_filters=1280
dropout_keep_prob=0.5
l2_reg_lambda=0.1
batch_size=128
num_epochs=20           ## 성능 개선 시, 유동적으로 조정
evaluate_every=100
checkpoint_every=100
allow_soft_placement= True # 세션의 설정옵션으로 GPU를 특정하지 않기
log_device_placement= False # 연산이 어느 디바이스로 설정되었는지 보여주지 않기

**학습 데이터 준비(데이터 로드 및 전처리)**

In [25]:
model = tf.global_variables_initializer()

# 학습데이터 경로
file_train_instances = '/content/mission1_train.csv'

# Load data
print("Loading data...")
x_heads, x_bodies, y = load_data_and_labels(file_train_instances)

Loading data...


In [26]:
# 모든 제목에 등장하는 단어들에 순서대로 인덱스를 할당(중복되는 단어는 새로운 인덱스가 아닌 이전의 동일한 인덱스 할당)
# 길이(단어 개수)가 다른 제목들에 대해, max_document_length 크기로 맞춰주는 역할
vocab_processor_head = learn.preprocessing.VocabularyProcessor(max_document_length=128)   # 객체 선언(사이즈 128)
x_head = np.array(list(vocab_processor_head.fit_transform(x_heads)))

# 모든 본문에 대해 위와 동일한 작업 수행
vocab_processor_body = learn.preprocessing.VocabularyProcessor(max_document_length=1280)
x_body = np.array(list(vocab_processor_body.fit_transform(x_bodies)))

print('----headline_shape----')
print(x_head.shape)
print('----body_shape----')
print(x_body.shape)
print('----label_shape----')
print(y.shape)


# 데이터를 랜덤하게 shuffle
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y))) # ex) len(y)=100 -> 0~99의 숫자를 무작위 순서로 반환

x_head_shuffled = x_head[shuffle_indices]
x_body_shuffled = x_body[shuffle_indices]
y_shuffled = y[shuffle_indices]


## 성능 개선 시, 유동적으로 조정
# 데이터 셋을 train/test set으로 나누기
# TODO: This is very crude, should use cross-validation
x_train_head, x_dev_head = x_head_shuffled[:-5000], x_head_shuffled[-5000:]
x_train_body, x_dev_body = x_body_shuffled[:-5000], x_body_shuffled[-5000:]
y_train, y_dev = y_shuffled[:-5000], y_shuffled[-5000:]

print("Vocabulary Size_head: {:d}".format(len(vocab_processor_head.vocabulary_)))  # 기사 제목에 있는 단어의 총 개수
print("Vocabulary Size_body: {:d}".format(len(vocab_processor_body.vocabulary_)))  # 기사 본문에 있는 단어의 총 개수
print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
print(x_train_head)
print(x_train_body)
print(x_train_head.shape)
print(x_train_body.shape)

W0623 08:03:44.636599 140091589158784 deprecation.py:323] From <ipython-input-26-5950458c1c8c>:1: VocabularyProcessor.__init__ (from tensorflow.contrib.learn.python.learn.preprocessing.text) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tensorflow/transform or tf.data.
W0623 08:03:44.638184 140091589158784 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/preprocessing/text.py:154: CategoricalVocabulary.__init__ (from tensorflow.contrib.learn.python.learn.preprocessing.categorical_vocabulary) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tensorflow/transform or tf.data.
W0623 08:03:44.641048 140091589158784 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/preprocessing/text.py:170: tokenizer (from tensorflow.contrib.learn.python.learn.preprocessing.text) is deprecated and will be removed in

----headline_shape----
(30140, 128)
----body_shape----
(30140, 1280)
----label_shape----
(30140, 2)
Vocabulary Size_head: 25604
Vocabulary Size_body: 72842
Train/Dev split: 25140/5000
[[ 5550  7119  5401 ...     0     0     0]
 [  691  1125  1329 ...     0     0     0]
 [ 2491 15880  2081 ...     0     0     0]
 ...
 [ 3978   997   896 ...     0     0     0]
 [  437 10969  3918 ...     0     0     0]
 [  294 16490  4226 ...     0     0     0]]
[[67292    84   114 ...     0     0     0]
 [ 7721  3805  6869 ...     0     0     0]
 [ 1615    26  1263 ...     0     0     0]
 ...
 [17448  1128 19474 ...     0     0     0]
 [16854   730 29922 ...     0     0     0]
 [ 2120   698  5109 ...     0     0     0]]
(25140, 128)
(25140, 1280)


**학습 시작**

In [36]:
# =======================================================
# 1. 텐서플로우 그래프 생성 이후 CNN 객체를 생성하기
# =======================================================

with tf.Graph().as_default(): # 세션을 새로 생성
    session_conf = tf.ConfigProto(allow_soft_placement=allow_soft_placement, log_device_placement=log_device_placement)
    sess = tf.Session(config=session_conf)
    # 세션 또한 Context manager를 사용하여 세션의 열고 닫는 처리를 자동으로 해준다
    
    with sess.as_default():
        # cnn 객체 생성
        cnn = Affine( 
            sequence_length_head=128, 
            sequence_length_body=1280, 
            num_classes=2, 
            vocab_size_head=len(vocab_processor_head.vocabulary_),
            vocab_size_body=len(vocab_processor_body.vocabulary_),
            embedding_size=embedding_dim, 
            filter_sizes=list(map(int,filter_sizes.split(","))),
            num_filters=num_filters,
            l2_reg_lambda=l2_reg_lambda
            )
        
        
# =========================================================
# 2. CNN모델을 학습하기 위한 학습명령(Optimizer) 지정
# =========================================================   

        # step count
        global_step = tf.Variable(0, name="global_step", trainable=False) 
    
        # 손실함수로 Adam Optimizer 사용
        optimizer = tf.train.AdamOptimizer(1e-3)  ##조정 가능?
        
        # cnn의 loss값을 파라미터로 받아 점진하강
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        
        # 학습에 사용할 함수 정의
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)
        

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
                
        grad_summaries_merged = tf.summary.merge(grad_summaries)


        # 모든 변수 초기화
        sess.run(tf.global_variables_initializer())
        
        # 학습데이터 워드 임베딩
        head_embedding = load_word_embedding("fasttext_3_10.vec",vocab_processor_head,embedding_dim)
        body_embedding = load_word_embedding("fasttext_3_10.vec",vocab_processor_body,embedding_dim)
        
        sess.run(cnn.embeddings_head.assign(head_embedding))
        sess.run(cnn.embeddings_body.assign(body_embedding))
        
# =========================================================
# 3-1. 학습의 단위를 Batch로 지정
# =========================================================           

        # train_step은 모델을 학습하는 하나의 묶음(batch)
        def train_step(x_batch_head, x_batch_body, y_batch):
            """
            A single training step
            """
          # 입력/예측 출력값을 넣어줌으로서 학습/평가를 할 수 있도록 함
          # overfiting을 줄이기 위해, dropout 확률을 지정
            feed_dict = {
              cnn.input_x_head: x_batch_head,
              cnn.input_x_body: x_batch_body,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: dropout_keep_prob 
            }
            
            # 설정 값들을 이용해 학습 시작
            _, step, loss, accuracy, predictions = sess.run([train_op, global_step, cnn.loss, cnn.accuracy, cnn.predictions], feed_dict)
            
            print("step {}, loss {:g}, acc {:g}".format(step, loss, accuracy))
            #print(predictions)
            
            
# =========================================================
# 3-2. 학습 평가의 단위를 Batch로 지정
# ========================================================= 

        #dev_step은 학습 결과 묶음(batch)를 평가
        def dev_step(x_batch_head, x_batch_body, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              cnn.input_x_head: x_batch_head,
              cnn.input_x_body: x_batch_body,
              cnn.input_y: y_batch,
              cnn.dropout_keep_prob: 1.0 #평가시에는 dropout은 사용하지 않는다(dropout_keep_prob:1.0=>off)
            }
            #평가시에는 학습용 train_op 파라미터는 넣지 않음
            step, loss, accuracy, predictions = sess.run([global_step, cnn.loss, cnn.accuracy, cnn.predictions], feed_dict)
            #반환
            return accuracy, loss, predictions
          
          
# ==========================================================
# 4. 전체 학습 세트를 배치단위로 나눔
# ==========================================================   
        # 한번 학습단위의 묶음은 batch_size/ 학습데이터는 num_epochs만큼 반복 사용
        batches = batch_iter(list(zip(x_train_head, x_train_body, y_train)), batch_size, num_epochs)   # zip(*iterable)은 동일한 개수로 이루어진 자료형을 묶어 주는 역할을 하는 함수
      
      
# ==========================================================
# 5. 전체 배치단위에 대해서 반복하며 학습 적용
# ==========================================================    

        for batch in batches:
    
            # zip을 사용하여 x(입력)과 y(기대출력)값을 각각 뽑아서
            x_batch_head, x_batch_body, y_batch = zip(*batch)
        
            # batch 단위로 학습 진행
            train_step(x_batch_head, x_batch_body, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            
            
            # 현재 학습 회차가 evaluate 할 순서이면 evalutate를 한 x_dev/ 기본 200번마
            if current_step % evaluate_every == 0:
                print("\nEvaluation:")
                
                # 평가용 데이터를 batch 단위로 가져옴
                batches_dev = batch_iter(list(zip(x_dev_head, x_dev_body, y_dev)),128, 1)
                acc_total, loss_total, total = 0,0,0
                for batch_dev in batches_dev:
                    x_batch_dev_head, x_batch_dev_body, y_batch_dev = zip(*batch_dev)
                    
                    # 학습된 모델에 평가용 batch 데이터를 넣어 acc와 loss를 누적함
                    acc_dev, loss_dev, predictions = dev_step(x_batch_dev_head, x_batch_dev_body, y_batch_dev)
                    acc_total = acc_total + acc_dev
                    loss_total = loss_total + loss_dev
                    #print(predictions)
                    total = total + 1
                    
                # 모델의 loss,acc 출력
                #results = codecs.open('results_all_new.txt','a')
                print("step {}, total loss:{}, total acc:{}".format(current_step, loss_total/total, acc_total/total))
                #results.write("total loss{:g}, total acc{:g}".format(loss_total/total, acc_total/total) + '\n')
                #results.close()
                print("")

(1, ?, 1, 1, 1280)
여기
(?, 1280)
(?, 1024)
22222
(?, 2)
%d/%d Tensor("output/predictions:0", shape=(?,), dtype=int64) Tensor("input_y:0", shape=(?, 2), dtype=float32)
Load word2vec file fasttext_3_10.vec

[[ 0.20702398 -0.02896587  0.22282847 ...  0.07108867  0.17335148
  -0.06750643]
 [-0.73611    -0.17625     0.24225    ... -0.40445     0.071112
   0.086732  ]
 [ 0.13608    -0.089695   -0.16513    ...  0.16671     0.18068
  -0.019453  ]
 ...
 [ 0.59938    -0.57996     0.096188   ... -0.49308    -0.19206
  -0.58257   ]
 [ 0.09263838  0.13136714  0.05774233 ...  0.00586049 -0.05260303
   0.02272747]
 [ 0.19149    -0.34612     0.18043    ...  0.37275     0.35202
  -0.20888   ]]
Load word2vec file fasttext_3_10.vec

[[ 0.11689787 -0.1111437   0.00791471 ...  0.07338314 -0.20677772
  -0.11932308]
 [-0.73611    -0.17625     0.24225    ... -0.40445     0.071112
   0.086732  ]
 [ 0.13608    -0.089695   -0.16513    ...  0.16671     0.18068
  -0.019453  ]
 ...
 [ 0.24050252 -0.14162     0.17603

# **모델 Test**

In [33]:
# Test 데이터 경로
file_test_instances = '/content/mission1_test.csv'

# Load data
print("Loading data...")
x_test_heads, x_test_bodies, y_test = load_data_and_labels_test(file_test_instances)

# 모든 제목에 등장하는 단어들에 순서대로 인덱스를 할당(중복되는 단어는 새로운 인덱스가 아닌 이전의 동일한 인덱스 할당)
# 길이(단어 개수)가 다른 제목들에 대해, max_document_length 크기로 맞춰주는 역할
vocab_processor_head_test = learn.preprocessing.VocabularyProcessor(max_document_length=128)   # 객체 선언(사이즈 128)
x_test_head = np.array(list(vocab_processor_head_test.fit_transform(x_test_heads)))

# 모든 본문에 대해 위와 동일한 작업 수행
vocab_processor_body_test = learn.preprocessing.VocabularyProcessor(max_document_length=1280)
x_test_body = np.array(list(vocab_processor_body_test.fit_transform(x_test_bodies)))

print('----headline_shape----')
print(x_test_head.shape)
print('----body_shape----')
print(x_test_body.shape)
print('----label_shape----')
print(y_test.shape)

print(x_test_head)
print(x_test_body)

Loading data...
----headline_shape----
(100, 128)
----body_shape----
(100, 1280)
----label_shape----
(100, 2)
[[  1   2   3 ...   0   0   0]
 [ 10  11  12 ...   0   0   0]
 [ 19  20  21 ...   0   0   0]
 ...
 [707 708 709 ...   0   0   0]
 [714 203 715 ...   0   0   0]
 [723 724 725 ...   0   0   0]]
[[   1    2    3 ...    0    0    0]
 [ 406  407  408 ...    0    0    0]
 [  11   12   13 ...    0    0    0]
 ...
 [ 679  137 4935 ...    0    0    0]
 [1427 5919 4541 ...    0    0    0]
 [ 679  137 1486 ...    0    0    0]]


In [38]:
batches_test = batch_iter(list(zip(x_test_head, x_test_body, y_test)),128, 1)

for batch_test in batches_test:
  x_batch_test_head, x_batch_test_body, y_batch_test = zip(*batch_test)
                    
  feed_dict = {
                  cnn.input_x_head: x_batch_test_head,
                  cnn.input_x_body: x_batch_test_body,
                  cnn.input_y: y_batch_test,
                  cnn.dropout_keep_prob: 1.0 #평가시에는 dropout은 사용하지 않는다(dropout_keep_prob:1.0=>off)
                }

  step, loss, accuracy, predictions = sess.run([global_step, cnn.loss, cnn.accuracy, cnn.predictions], feed_dict)
  print("Accuracy {:g}".format(accuracy))

Accuracy 0.47
